# Mikkel vs Hopfield and Krotov

In [2]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
from tqdm.notebook import tqdm_notebook as tqdm
from collections import defaultdict
from torchvision import datasets, transforms

In [7]:
#num_pixel = x_train.shape[1] * x_train.shape[2] * x_train.shape[3] #3072
#num_train = x_train.shape[0]
#num_test = x_test.shape[0]
#x_train_flat = x_train.reshape(num_train, num_pixel)
num_pixel=32*32*3
eps0 = 2e-2    # learning rate
Kx = 50
Ky = 50
num_hidden = Kx * Ky    # number of hidden units that are displayed in Ky by Kx array
mu = 0.0
sigma = 1.0
num_epochs = 1000      # number of epochs
num_batch = 1000      # size of the minibatch
prec = 1e-30
delta = 0.4    # Strength of the anti-hebbian learning
p = 2.0        # Lebesgue norm of the weights
k = 2          # ranking parameter, must be integer that is bigger or equal than 2

## Mikkel

In [11]:
def synaptic_activation(synapses, inputs):
    return (synapses.sign() * synapses.abs() ** (p - 1)).matmul(inputs)

def learning_activation(indices):
    best_ind, best_k_ind = indices[0], indices[k-1]
    g_i = torch.zeros(num_hidden, num_batch).to(device)
    g_i[best_ind,   torch.arange(num_batch).to(device)] = 1.0
    g_i[best_k_ind, torch.arange(num_batch).to(device)] = -delta
    return g_i

def weight_update(synapses, tot_input, inputs, g_i):
    xx = (g_i * tot_input).sum(dim=1)
    ds = torch.matmul(g_i, inputs.t()) - xx.unsqueeze(1) * synapses
        
    nc = max(ds.abs().max(), prec)
        
    # the actual update
    return eps * ds / nc

synapses_Mikkel = torch.Tensor(num_hidden, num_pixel).normal_(mu, sigma).to(device)

## Hopfield

In [12]:
def g(tot_input, delta=0.4, k=2):
    y=np.argsort(tot_input,axis=0)
    yl=np.zeros((hid,Num), dtype=np.double)
    yl[y[hid-1,:],np.arange(Num)]=1.0
    yl[y[hid-k],np.arange(Num)]=-delta
    return yl

def bracket(synapses, inputs, p=3):
    sig=np.sign(synapses)
    return np.dot(sig*np.absolute(synapses)**(p-1),inputs)

def ds(yl, tot_input):
    xx=np.sum(np.multiply(yl,tot_input),1)
    ds=np.dot(yl,np.transpose(inputs)) - np.multiply(np.tile(xx.reshape(xx.shape[0],1),(1,N)),synapses)

    nc=np.amax(np.absolute(ds))
    if nc<prec:
        nc=prec
    return np.true_divide(ds,nc)

In [13]:
synapses_Hopfield = synapses_Mikkel.clone()

## Comparison

In [18]:
Sinp = torch.rand(100, num_pixel).T

In [19]:
tot_inp_Mikkel = synaptic_activation(synapses_Mikkel, Sinp.to(device))

In [20]:
tot_inp_Hopfield = bracket(synapses_Hopfield.detach().cpu().numpy(), Sinp.detach().cpu().numpy())

In [21]:
# define a distance measure

def LpDist(x, y, p=2.):
    x = torch.Tensor(x)
    y = torch.Tensor(y)
    d = (x - y)**p
    return d.sum()

def is_zero_tensor(x, p=1., atol=1e-4):
    x = torch.Tensor(x)
    d = torch.abs(x)**p
    is_zero = (d - atol) < 0
    return is_zero.all()

In [27]:
with torch.no_grad():
    print(LpDist(tot_inp_Mikkel.cpu(), tot_inp_Hopfield))

tensor(2.0658e+08)
